# Libraries

In [ ]:
# pyspark
!pip install pyspark

# findspark
!pip install findspark

# jupyterlab
!pip install jupyterlab

# unicode
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=da453ebd504f6360fecfd577b87f67fbc7b1c9adc7f8374f379c343257eb7ed5
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 9.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import time
import pandas as pd

In [ ]:
import findspark
findspark.init()

import requests
import json
import unidecode
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()

#IBGE API

In [ ]:
ibge = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios").json()

In [ ]:
ibge

[{'id': 3502507,
  'nome': 'Aparecida',
  'microrregiao': {'id': 35051,
   'nome': 'Guaratinguetá',
   'mesorregiao': {'id': 3513,
    'nome': 'Vale do Paraíba Paulista',
    'UF': {'id': 35,
     'sigla': 'SP',
     'nome': 'São Paulo',
     'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}},
  'regiao-imediata': {'id': 350052,
   'nome': 'Guaratinguetá',
   'regiao-intermediaria': {'id': 3511,
    'nome': 'São José dos Campos',
    'UF': {'id': 35,
     'sigla': 'SP',
     'nome': 'São Paulo',
     'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}}},
 {'id': 3503158,
  'nome': 'Arapeí',
  'microrregiao': {'id': 35052,
   'nome': 'Bananal',
   'mesorregiao': {'id': 3513,
    'nome': 'Vale do Paraíba Paulista',
    'UF': {'id': 35,
     'sigla': 'SP',
     'nome': 'São Paulo',
     'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}},
  'regiao-imediata': {'id': 350053,
   'nome': 'Cruzeiro',
   'regiao-intermediaria': {'id': 3511,
    'nome': 'São José dos Campos',
   

In [ ]:
# fist try/way to save data from API IBGE in lists
id_city = []
name_city = []
name_micro_region = []
name_messo_region = []
Abbreviation_state = []
name_state = []
Abbreviation_region = []
name_region = []


for i in range(len(ibge)):
    id_city.append(ibge[i]['id'])
    name_city.append(ibge[i]['nome'])
    name_micro_region.append(ibge[i]['microrregiao']['nome'])
    name_messo_region.append(ibge[i]['microrregiao']['mesorregiao']['nome'])
    Abbreviation_state.append(ibge[i]['microrregiao']['mesorregiao']['UF']['sigla'])
    name_state.append(ibge[i]['microrregiao']['mesorregiao']['UF']['nome'])
    Abbreviation_region.append(ibge[i]['microrregiao']['mesorregiao']['UF']['regiao']['sigla'])
    name_region.append(ibge[i]['microrregiao']['mesorregiao']['UF']['regiao']['nome'])

In [ ]:
# second way to save data from IBGE api in one list just
list_df_ibge = []

for i in range(len(ibge)):
    list_df_ibge.append([ibge[i]['id'],
                        ibge[i]['nome'],
                        ibge[i]['microrregiao']['nome'],
                        ibge[i]['microrregiao']['mesorregiao']['nome'],
                        ibge[i]['microrregiao']['mesorregiao']['UF']['sigla'],
                        ibge[i]['microrregiao']['mesorregiao']['UF']['nome'],
                        ibge[i]['microrregiao']['mesorregiao']['UF']['regiao']['sigla'],
                        ibge[i]['microrregiao']['mesorregiao']['UF']['regiao']['nome']])


In [ ]:
list_df_ibge

[[3502507,
  'Aparecida',
  'Guaratinguetá',
  'Vale do Paraíba Paulista',
  'SP',
  'São Paulo',
  'SE',
  'Sudeste'],
 [3503158,
  'Arapeí',
  'Bananal',
  'Vale do Paraíba Paulista',
  'SP',
  'São Paulo',
  'SE',
  'Sudeste'],
 [3503505,
  'Areias',
  'Bananal',
  'Vale do Paraíba Paulista',
  'SP',
  'São Paulo',
  'SE',
  'Sudeste'],
 [3504909,
  'Bananal',
  'Bananal',
  'Vale do Paraíba Paulista',
  'SP',
  'São Paulo',
  'SE',
  'Sudeste'],
 [3508504,
  'Caçapava',
  'São José dos Campos',
  'Vale do Paraíba Paulista',
  'SP',
  'São Paulo',
  'SE',
  'Sudeste'],
 [3508603,
  'Cachoeira Paulista',
  'Guaratinguetá',
  'Vale do Paraíba Paulista',
  'SP',
  'São Paulo',
  'SE',
  'Sudeste'],
 [3509700,
  'Campos do Jordão',
  'Campos do Jordão',
  'Vale do Paraíba Paulista',
  'SP',
  'São Paulo',
  'SE',
  'Sudeste'],
 [3509957,
  'Canas',
  'Guaratinguetá',
  'Vale do Paraíba Paulista',
  'SP',
  'São Paulo',
  'SE',
  'Sudeste'],
 [3510500,
  'Caraguatatuba',
  'Caraguatatuba

In [ ]:
#create dataframe with list from IBGE API
df_ibge = pd.DataFrame(list_df_ibge, columns=['id_city','name_city','name_micro_region','name_messo_region',
                                                 'Abbreviation_state','name_state','Abbreviation_region','name_region'])

In [ ]:
df_ibge

,id_city,name_city,name_micro_region,name_messo_region,Abbreviation_state,name_state,Abbreviation_region,name_region
0,3502507,Aparecida,Guaratinguetá,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
1,3503158,Arapeí,Bananal,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
2,3503505,Areias,Bananal,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
3,3504909,Bananal,Bananal,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
4,3508504,Caçapava,São José dos Campos,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
5,3508603,Cachoeira Paulista,Guaratinguetá,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
6,3509700,Campos do Jordão,Campos do Jordão,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
7,3509957,Canas,Guaratinguetá,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
8,3510500,Caraguatatuba,Caraguatatuba,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
9,3513405,Cruzeiro,Guaratinguetá,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste


#Weather API

In [ ]:
#df without 3 cities - that we can't find in the weather api
df_ibge2 = df_ibge.drop(index=[11,12,24]).reset_index(drop=True)

In [ ]:
# code to get data from weather api and create a dataframe
link = 'http://api.weatherapi.com/v1/forecast.json?key=eec58040ec21402cabe173206232203&q=São Paulo&days=6&aqi=no&alerts=no'
columns_df = requests.get(link).json()
list_weather_keys = []
for key, value in columns_df['location'].items(): 
    list_weather_keys.append(key)
for i in range(6):
    list_weather_keys.append('date'+str(i))
    for key, value in columns_df['forecast']['forecastday'][0]['day'].items():
        list_weather_keys.append(key+"day"+str(i))
df_weather = pd.DataFrame(columns = [list_weather_keys])

for name_city in df_ibge2.name_city	:
  http_weather = 'http://api.weatherapi.com/v1/forecast.json?key=eec58040ec21402cabe173206232203&q='+name_city+'&days=6&aqi=no&alerts=no'
  weather_request = requests.get(http_weather).json()
  list_weather_value = []
  for key, value in weather_request['location'].items(): 
        list_weather_value.append(value)
  for i in range(6):
      list_weather_value.append(weather_request['forecast']['forecastday'][0]['date'])
      for key,value in columns_df['forecast']['forecastday'][i]['day'].items():
              if key == "condition":
                  list_weather_value.append(columns_df['forecast']['forecastday'][i]['day']['condition']['text'])
              else:
                  list_weather_value.append(value)       
  df_weather.loc[len(df_weather)] = list_weather_value
  

In [ ]:
df_weather['daily_will_it_rainday0'] = df_weather['daily_will_it_rainday0'].replace({0: 'não', 1: 'sim'})  

In [ ]:
df_weather

,name,region,country,lat,lon,tz_id,localtime_epoch,localtime,date0,maxtemp_cday0,...,totalsnow_cmday5,avgvis_kmday5,avgvis_milesday5,avghumidityday5,daily_will_it_rainday5,daily_chance_of_rainday5,daily_will_it_snowday5,daily_chance_of_snowday5,conditionday5,uvday5
0,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1679676543,2023-03-24 13:49,2023-03-24,33.6,...,0.0,8.0,4.0,85.0,1,85,0,0,Moderate rain,5.0
1,Arapeí,Sao Paulo,Brasil,-22.68,-44.45,America/Sao_Paulo,1679676543,2023-03-24 13:49,2023-03-24,33.6,...,0.0,8.0,4.0,85.0,1,85,0,0,Moderate rain,5.0
2,Areias,Paraiba,Brazil,-6.45,-38.43,America/Fortaleza,1679676544,2023-03-24 13:49,2023-03-24,33.6,...,0.0,8.0,4.0,85.0,1,85,0,0,Moderate rain,5.0
3,Bananal,Maranhao,Brazil,-3.99,-43.12,America/Fortaleza,1679676544,2023-03-24 13:49,2023-03-24,33.6,...,0.0,8.0,4.0,85.0,1,85,0,0,Moderate rain,5.0
4,Caçapava,Sao Paulo,Brasil,-23.10,-45.72,America/Sao_Paulo,1679676545,2023-03-24 13:49,2023-03-24,33.6,...,0.0,8.0,4.0,85.0,1,85,0,0,Moderate rain,5.0
5,Cachoeira Paulista,Sao Paulo,Brazil,-22.67,-45.02,America/Sao_Paulo,1679676546,2023-03-24 13:49,2023-03-24,33.6,...,0.0,8.0,4.0,85.0,1,85,0,0,Moderate rain,5.0
6,Campos,Viana do Castelo,Portugal,41.97,-8.70,Europe/Lisbon,1679676547,2023-03-24 16:49,2023-03-24,33.6,...,0.0,8.0,4.0,85.0,1,85,0,0,Moderate rain,5.0
7,Canas,Guanacaste,Costa Rica,10.43,-85.10,America/Costa_Rica,1679676547,2023-03-24 10:49,2023-03-24,33.6,...,0.0,8.0,4.0,85.0,1,85,0,0,Moderate rain,5.0
8,Caraguatatuba,Sao Paulo,Brazil,-23.62,-45.42,America/Sao_Paulo,1679676548,2023-03-24 13:49,2023-03-24,33.6,...,0.0,8.0,4.0,85.0,1,85,0,0,Moderate rain,5.0
9,Cruzeiro Do Sul,Acre,Brazil,-7.63,-72.60,America/Rio_Branco,1679676549,2023-03-24 11:49,2023-03-24,33.6,...,0.0,8.0,4.0,85.0,1,85,0,0,Moderate rain,5.0


#Spark

In [ ]:
# create a spark df
df_spark_weather = spark.createDataFrame(df_weather,list_weather_keys)
df_spark_weather.show()

+-------------------+----------------+----------+------+------+------------------+---------------+----------------+----------+-------------+-------------+-------------+-------------+-------------+-------------+---------------+---------------+------------------+------------------+----------------+-------------+----------------+---------------+----------------------+------------------------+----------------------+------------------------+--------------------+------+----------+-------------+-------------+-------------+-------------+-------------+-------------+---------------+---------------+------------------+------------------+----------------+-------------+----------------+---------------+----------------------+------------------------+----------------------+------------------------+--------------------+------+----------+-------------+-------------+-------------+-------------+-------------+-------------+---------------+---------------+------------------+------------------+--------------

In [ ]:
df_spark_ibge = spark.createDataFrame(df_ibge2)
df_spark_ibge.show()

+-------+-------------------+--------------------+--------------------+------------------+----------+-------------------+-----------+
|id_city|          name_city|   name_micro_region|   name_messo_region|Abbreviation_state|name_state|Abbreviation_region|name_region|
+-------+-------------------+--------------------+--------------------+------------------+----------+-------------------+-----------+
|3502507|          Aparecida|       Guaratinguetá|Vale do Paraíba P...|                SP| São Paulo|                 SE|    Sudeste|
|3503158|             Arapeí|             Bananal|Vale do Paraíba P...|                SP| São Paulo|                 SE|    Sudeste|
|3503505|             Areias|             Bananal|Vale do Paraíba P...|                SP| São Paulo|                 SE|    Sudeste|
|3504909|            Bananal|             Bananal|Vale do Paraíba P...|                SP| São Paulo|                 SE|    Sudeste|
|3508504|           Caçapava| São José dos Campos|Vale do Para

In [ ]:
df_spark_weather.createOrReplaceTempView('forecast')
df_spark_ibge.createOrReplaceTempView('cities')

In [ ]:
spark.sql("select id_city, name_city from cities").show()

+-------+-------------------+
|id_city|          name_city|
+-------+-------------------+
|3502507|          Aparecida|
|3503158|             Arapeí|
|3503505|             Areias|
|3504909|            Bananal|
|3508504|           Caçapava|
|3508603| Cachoeira Paulista|
|3509700|   Campos do Jordão|
|3509957|              Canas|
|3510500|      Caraguatatuba|
|3513405|           Cruzeiro|
|3513603|              Cunha|
|3520400|           Ilhabela|
|3524402|            Jacareí|
|3524907|           Jambeiro|
|3526308|           Lagoinha|
|3526605|          Lavrinhas|
|3527207|             Lorena|
|3531704|    Monteiro Lobato|
|3532306|Natividade da Serra|
|3535606|          Paraibuna|
+-------+-------------------+
only showing top 20 rows



In [ ]:
spark.sql("select name, country from forecast").show()

+-------------------+----------+
|               name|   country|
+-------------------+----------+
|          Aparecida|    Brazil|
|             Arapeí|    Brasil|
|             Areias|    Brazil|
|            Bananal|    Brazil|
|           Caçapava|    Brasil|
| Cachoeira Paulista|    Brazil|
|             Campos|  Portugal|
|              Canas|Costa Rica|
|      Caraguatatuba|    Brazil|
|    Cruzeiro Do Sul|    Brazil|
|              Cunha|    Brazil|
|           Ilhabela|    Brazil|
|            Jacareí|    Brasil|
|           Jambeiro|    Brazil|
|           Lagoinha|    Brazil|
|          Lavrinhas|    Brazil|
|             Lorena|    Brazil|
|    Monteiro Lobato|    Brazil|
|Natividade Da Serra|    Brazil|
|          Paraibuna|    Brazil|
+-------------------+----------+
only showing top 20 rows



In [ ]:
spark.sql("select forecast.name, cities.name_state ,forecast.country from forecast join cities on forecast.name = cities.name_city ").show()

+------------------+----------+----------+
|              name|name_state|   country|
+------------------+----------+----------+
|         Aparecida| São Paulo|    Brazil|
|            Arapeí| São Paulo|    Brasil|
|            Areias| São Paulo|    Brazil|
|           Bananal| São Paulo|    Brazil|
|Cachoeira Paulista| São Paulo|    Brazil|
|             Canas| São Paulo|Costa Rica|
|     Caraguatatuba| São Paulo|    Brazil|
|          Caçapava| São Paulo|    Brasil|
|             Cunha| São Paulo|    Brazil|
|          Ilhabela| São Paulo|    Brazil|
|           Jacareí| São Paulo|    Brasil|
|          Jambeiro| São Paulo|    Brazil|
|          Lagoinha| São Paulo|    Brazil|
|         Lavrinhas| São Paulo|    Brazil|
|            Lorena| São Paulo|    Brazil|
|   Monteiro Lobato| São Paulo|    Brazil|
|         Paraibuna| São Paulo|    Brazil|
|   Pindamonhangaba| São Paulo|    Brazil|
|           Piquete| São Paulo|    Brazil|
|            Queluz| São Paulo|  Portugal|
+----------

In [ ]:
spark.sql('SELECT f.name, \
                  c.id_city, \
                  f.localtime, \
                  f.date0, \
                  c.name_region, \
                  c.name_state, \
                  f.country, \
                  f.lat, \
                  f.lon, \
                  f.maxtemp_cday0, \
                  f.mintemp_cday0, \
                  f.avgtemp_cday0, \
                  f.daily_will_it_rainday0, \
                  f.daily_chance_of_rainday0, \
                  f.conditionday0, \
                  f.maxwind_mphday0 \
          FROM forecast as f \
          JOIN cities as c \
          ON f.name = c.name_city').show()















+------------------+-------+----------------+----------+-----------+----------+----------+------+------+-------------+-------------+-------------+----------------------+------------------------+--------------------+---------------+
|              name|id_city|       localtime|     date0|name_region|name_state|   country|   lat|   lon|maxtemp_cday0|mintemp_cday0|avgtemp_cday0|daily_will_it_rainday0|daily_chance_of_rainday0|       conditionday0|maxwind_mphday0|
+------------------+-------+----------------+----------+-----------+----------+----------+------+------+-------------+-------------+-------------+----------------------+------------------------+--------------------+---------------+
|         Aparecida|3502507|2023-03-24 13:49|2023-03-24|    Sudeste| São Paulo|    Brazil|-22.83|-45.23|         33.6|         19.7|         25.1|                   sim|                      87|Patchy rain possible|            9.2|
|            Arapeí|3503158|2023-03-24 13:49|2023-03-24|    Sudeste| São

In [ ]:
spark.sql('SELECT f.name, \
                  \
          FROM forecast as f \
          JOIN cities as c \
          ON f.name = c.name_city').show()